<img src="https://datasciencecampus.ons.gov.uk/wp-content/uploads/sites/10/2017/03/data-science-campus-logo-new.svg"
             alt="ONS Data Science Campus Logo"
             width = "240"
             style="margin: 0px 60px"
             />

# 1.0 Exploratory Data Analysis on Titanic Training Data

## Introduction

Purpose of script: exploratory data analysis of titanic test dataset, feature engineering
source: [kaggle](https://www.kaggle.com/c/titanic/data)


<div class="row">
    <div class="column">
        <img src="../../images/photo-1534685785745-60a2cea0ec34.jpeg"
             alt="Photo of explorer stood atop mountain from https://unsplash.com/photos/8oYPewvmhnY"
             align="center" 
             width = "500"
             />
    </div>

Note that the feature engineering used here differs to the guide provided by Kaggle. You are welcome to use the engineered features outlined in this script, from the guide and any ideas of your own. Please feel free to experiment and see if you can improve the performance of the model beyond that achieved here.

[Guide to feature engineering titanic data](https://triangleinequality.wordpress.com/2013/09/08/basic-feature-engineering-with-the-titanic-data/)

Kaggle have also published a guide to machine learning and submission of output with this data set. [kaggle guide](https://www.kaggle.com/alexisbcook/titanic-tutorial)

In [ ]:
# import necessary libraries
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
# import test data
titanic_train = pd.read_csv('../../data/train.csv')
titanic_test = pd.read_csv('../../data/test.csv')
# cache unedited df for use in training later
pd.to_pickle(titanic_train, '../../cache/titanic_train.pkl')
pd.to_pickle(titanic_test, '../../cache/titanic_test.pkl')

## Data Dictionary

There are a number of variables within this dataset:
* pclass = Passenger class of travel.
* survived = 1 if the passenger survived the sinking, 0 if not.
* name = Full name of the passenger, including title.
* sex = Passenger gender.
* age = Passenger age.
* sibsp = Count of siblings or spouse also aboard.
* Parch = Count of parents or children also aboard.
* ticket = Ticket reference.
* fare = Fare paid.
* cabin = Cabin number.
* embarked = Port of embarkation. (S = Southampton (UK); C = Cherbourg (France); Q = Queenstown (Cobh, Ireland))

Note that the **titanic_test** data contains the features but no target, i.e. it does not include the **Survived** column. It is expected that once you have achieved the desired model performance, that you will then predict labels upon the test data and submit to Kaggle, who will provide you with a performance report.

In [ ]:
#adjust console presentation of output
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)



## Summary Statistics

In [ ]:
#dimensions of dataframe
titanic_train.shape

In [ ]:
#summary statistics of numeric columns
titanic_train.describe()


Some initial thoughts: 

• PassengerId, is it unique?  
• Survived, can see from quartiles that majority did not survive. Would hope that the test DF would be based on the same distribution.  
• Pclass, relatively fewer 1st class passengers. Majority are 3rd class.   
• Age, floating point. Estimated ages include a .5 value. There is a degree of missingness, only 714 observations. What proportion is missing? What proportion is estimated?  
• Sibsp, number of siblings or spouses aboard. Mistresses and fiances were ignored. Majority recorded having no siblings. Mean is higher than median, large outliers influenced? Max is 8 siblings.  
• Parch, number of parents / children on board. Children travelling with nanny only Parch == 0. Large outlier influence on mean as in SibSp.  
• Fare, minimum is zero. How many? Are these the crew? Do they have a unique pclass and age profile? Mean is double the median, high large outlier influence. Maximum is £512, extremely high for 1912 context.  


In [ ]:
# Class imbalance in target suggested. Tally values. Do not drop NAs if present and return percentages of classes.
titanic_train.Survived.value_counts(dropna=False, normalize=True)


Just under 40% survived. So a class imbalance exists but I wouldn't say this is a candidate for random over sampling.

In [ ]:
# summary stats for object columns
titanic_train.describe(include='object')


Some observations:

* Name column has no duplicates, I happen to know the full dataset has 2 non unique values. Either in test df or split. Likely of no consequence for this analysis.  
* Sex appears to be mostly male. Look at proportion male:female. 
* ticket ref is object, column is a mix of character / numeric values. Interestingly, 230 values are non-unique - family tickets?
* Cabin number has a high degree of missingness. Only 204 values. This is a shame as it would have been very interesting to analyse survival based on distance from the deck level. Perhaps isolate this group and investigate?
* Embarked - port of embarkation has 3 values. Most common one is S = Southampton.  2 missing values here.


Some questions have been identified to pursue further. My main concern so far would be in regards to the method of splitting the test dataset. No detail on how this has been done is provided and hopefully it is not not just based on the distribution of the survived column alone. 



## Pairwise Correlation

In [ ]:
# explore the pairwise correlations within the data using pearson's correlation coefficient
titanic_train.corr(method ='pearson') 

Some interesting trends observed, Only Parch & Fare indicate a positive correlation with Survival, neither of which would be indicated as strong (defined as corr of +/-0.7 or higher) all other numeric feautures indicate weak negative correlations against survived suggests a lot of noise in observed trends in features against survival. Weak though they are, the largest coefficients observed (+ or -) against survived are Pclass & Fare. Within the matrix as a whole, Pclass & Fare has the strongest observed coefficient of -0.549500 Not a strong correlation though (defined as +/-0.7 threshold). A moderate negative correlation. The next strongest correlation would be SibSp & Parch with a weak positive correlation of 0.414838.

***

## Feature Analysis

### PassengerId

In [ ]:
# is it unique?
titanic_train.PassengerId.value_counts(dropna=False)
# appears so.

***

### Survived

In [ ]:
########Survived###########
# distribution?
titanic_train.Survived.value_counts(normalize=True)
# 62% didn't survive. Will check this against the test set. 


***
### Pclass

In [ ]:
# Plot a histogram with 3 bins to view the distribution of this feature.
plt.hist(titanic_train.Pclass, bins=3)

3rd class is by far the most common pclass. Interesting that there appears to be more 1st than 2nd class.


In [ ]:
titanic_train.Pclass.value_counts(normalize=True) 

***
### Age

In [ ]:
########Age###########
titanic_train.Age.value_counts(dropna=False, normalize=True)

**NaN 0.198653.** Nearly 20% missing data. Parch / Sibsp won't offer the required directionality to help inform imputation here, as I understand it.

what proportion are estimated *and* equal to 1 or older?

In [ ]:
#filter data where age is greater than one and a whole number (ages not estimated). Divide by the totals rows.
len(titanic_train.query('Age >= 1.0 & Age % 1 == 0.5')) / len(titanic_train)

2 percent of data are estimated age. Combined with #NAs, this makes for a column with significant quality issues. This is something that could be useful to record in a flag.

In [ ]:
# Create a feature that records estimated ages: age_estimated
titanic_train.insert(
     #add this column after the Age column
    (titanic_train.columns.get_loc("Age") + 1),
    # wherever age is estimated, assign True, otherwise False. 
    "age_estimated", np.where((titanic_train.Age >= 1.0) & (titanic_train.Age % 1 == 0.5), True, False)
)

We can also introduce age bins in order to explore differential survival rates. School age from [wikipedia](https://en.wikipedia.org/wiki/Raising_of_school_leaving_age_in_England_and_Wales#19th_century)

In [ ]:
# define labels for the age categories
cut_labels = ['pre_school', 'school', 'adolescent', 'age_of_majority', 'pensioners']
# define boundary limits for the boundaries
cut_bins = [0, 4.5, 13.5, 20.5, 69.5, 80]

In [ ]:
# insert an age quantile column to investigate differential survival
titanic_train.insert(
     # Find the location of the Age column and insert the new column after it
     (titanic_train.columns.get_loc("Age") + 1),
     # discretize the Age values using the bins and labels provided above.
     "age_discrete", value=pd.cut(titanic_train.Age, bins=cut_bins, labels=cut_labels)
)

In [ ]:
# tidy up
del cut_bins, cut_labels



Investigate differential survival among the age categories, interesting to look at pre school survival rates. Create a dataframe containing only preschool-age passengers.

In [ ]:
preschool = titanic_train.query('age_discrete == "pre_school"')

In [ ]:
# eda on the preschool dataset. Print a correlation matrix.
preschool.corr(method ='pearson')


Interesting that a strong negative correlation is returned for SibSp x Survived -0.714502 . Could this indicate that passengers of smaller families (with preschoolers) were more likely to survive than individuals with larger families?

In [ ]:
preschool.Survived.value_counts(normalize=True)

In [ ]:
# How many observations are within this dataframe?
len(preschool)


Small number of observations, high correlation observed could be a relic of the data. As this correlation was not observed within total df, the other age categories must be responsible for the overall trend. Perhaps school age? Create the school age df in the same way.

***

In [ ]:
# filter titanic_train for rows that are categorized as "school" for the discretized age column.
schoolchildren = titanic_train.query('age_discrete == "school"')
# Print a pariwise correlation matrix
schoolchildren.corr(method='pearson')


In [ ]:
len(schoolchildren)

Again, a small subset. Moderate -ve correlation between SibSp & survived, -0.668927.

There appears to be a more pronounced trend in pclass x survived here -0.526789 than in preschool -0.325941. Both small subsets however.

In [ ]:
schoolchildren.Survived.value_counts(normalize=True)

Survival in this age group appears to have declined in comparison to preschool. 

***

What about adolescent passengers?

In [ ]:
adolescents = titanic_train.query('age_discrete == "adolescent"')
adolescents.corr(method='pearson')
# negligible correlation with SibSp x survived in this subset. -0.054019 
# moderate correlation between PClass x survived -0.439298, appears to be obeying law of central limit theorem/

In [ ]:
len(adolescents)

More observations in this age group. Could be that a moderate trend detected between PClass & survival could hold more weight.

In [ ]:
# adolescent survival by gender - plot a bar chart
adolescents.groupby('Sex').Survived.value_counts().plot.bar()


In [ ]:
# Generate normalised counts of the values diplayed in the above bar chart.
adolescents.groupby('Sex').Survived.value_counts(normalize=True)

Clear distinction in survival rate by sex in adolescent-aged group.

In [ ]:
# adolescent survival by class?
adolescents.groupby('Pclass').Survived.value_counts().plot.bar()

Visually, the survival differential is in favour of the 1st class adolescents


In [ ]:
adolescents.groupby('Pclass').Survived.value_counts(normalize=True)


Could be an interesting candidate for chi2 test. Difference between the classes appears worhwhile investigating.

In [ ]:
# create a crosstab to analyse
tab = pd.crosstab(adolescents.Pclass, adolescents.Survived)
print(tab)

Unable to run chi2 as there is an observed frequency in the contingency table of below 5. Assumption would be violated.

In [ ]:
# tidy up
del adolescents, preschool, schoolchildren, tab

***

### Sibsp

In [ ]:
# reset the plot canvas
plt.clf()
# distribution? plot a histogram of the SibSp feature
plt.hist(titanic_train.SibSp)


Small number of observations at 8 sibsp skewing the mean over the median. Frequency for these?

In [ ]:
titanic_train.SibSp.value_counts(dropna=False)

Interesting that ```sibsp == 8``` returns a count of 7. The missing 1 must be in the test df?

In [ ]:
# reset the plot canvas
plt.clf()
# plot a chart that allows visual identification of outliers in the SibSp feature
plt.boxplot(titanic_train.SibSp)


Chart is flagging sibsp values > 2 as outliers. This coincides with 1.5 * inter quartile range (outlier threshold dthen defined as > 2.5). Would see no valid reason to omit these outliers, but to be aware that this is a column with significant left skew. 

***

### Parch

In [ ]:
# reset the plot canvas
plt.clf()
# distribution?
plt.hist(titanic_train.Parch)

Some large values with low frequencies here.


In [ ]:
titanic_train.Parch.value_counts(dropna=False)

This result indicates an imbalance in train test split. 1 observation for Parch for a value of 6. The remaining 5 must be in the test group. These are all likely to qualify as outliers. Implications for logistic regression model training vs test. Possibly a candidate for outlier removal, but see no legitimate reason for doing so apart from improving model accuracy for parch rows < 6

In [ ]:
# reset the plot canvas
plt.clf()
# outlier analysis
plt.boxplot(titanic_train.Parch)

Boxplot flags everything above a 0 as outlier. Coincides with 1.5 * IQR. Families were in the minority and most of the families on board were small ones.

***

### Fare

In [ ]:
# reset the plot canvas
plt.clf()
#distribution
plt.hist(titanic_train.Fare, bins=20)

Previously established minimum fare is 0. Idea being that these were crew? From this histogram it is clear a significant prop paid little or nothing.

In [ ]:
titanic_train.Fare.value_counts(dropna=False)

Only 15 passengers paid 0 pounds. So this wouldn't represent a total crew. Let's investigate the zero fares further.


In [ ]:
# Create a dataframe 'zero_fares' by slicing titanic_train to wherever the passenger paid zero for Fare
zero_fares = titanic_train.query('Fare == 0')
# print summary statistics of all numeric columns for this dataframe
zero_fares.describe()

In [ ]:
zero_fares.describe(include='object')

Inspection of the df, all male, all embarked at Southampton, 1 survived, a mixture of classes. 4 have 'LINE' as ticket reference.  All have 0 family on board. All but 3 have no cabin number recorded. I wonder how often 'LINE' occurs in the total training set?


In [ ]:
titanic_train.query("Ticket == 'LINE'")  

No, this is not more widely recorded than within this group. There's not a great deal more to be said about this group without additional info. 

In [ ]:
#clean up
del zero_fares

***

### Sex

In [ ]:
# What is the gender ratio of the dataset?
titanic_train.Sex.value_counts(normalize=True)


Overall 65% male. Did 'women and children go first' make a difference here? 

In [ ]:
titanic_train.groupby(titanic_train.Sex).Survived.value_counts(normalize=True)

Although the gender imbalance is evident, ~3/4 females survived, whereas ~1/5 males survived. 

In [ ]:
titanic_train.groupby(titanic_train.Sex).Survived.value_counts().plot.bar()

Gender would be an important feature for supervised learning. 

***

### Ticket

In [ ]:
# add a duplicated tickets column
titanic_train.insert(
    # add column following the Ticket column
    (titanic_train.columns.get_loc("Ticket") + 1),
    # tally ticket frequencies (note that this may not be accurate due to hold out set)
    "duplicated_tickets", titanic_train.groupby('Ticket')['Ticket'].transform('count')
)

In [ ]:
# isolate all rows that show duplicated tickets
duplicated_tickets = titanic_train.query("duplicated_tickets > 1")
duplicated_tickets.Ticket.value_counts()


In [ ]:
duplicated_tickets.describe()

Interesting here that there are 0s for SibSp & also for Parch separately. Are there any for both columns?


In [ ]:
# Are there any rows in duplicated_tickets that show zero values for both SibSp & Parch?
len(duplicated_tickets.query('SibSp == 0 & Parch == 0'))

Yes, 75 of the duplicated tickets were 0 for both family indicators. Metadata indicates that these could be mistresses, fiancees and nannies. 20% of passengers with duplicated tickets are 0 for both family indicators

In [ ]:
duplicated_tickets.Sex.value_counts(normalize=True)

Interesting that in this subset the gender balance is slightly in favour of females. This contrasts the training df as a whole.


In [ ]:
duplicated_tickets.groupby('Sex').Survived.value_counts(normalize=True)

Closer to 20% males survived here and female survival rate looks the same as larger group. No apparent radical departure. No further question springs to mind on this with the data to hand. 

In [ ]:
# Clean up 
del duplicated_tickets

***

### Cabin number

In [ ]:
# value counts with NA
titanic_train.Cabin.value_counts(dropna=False)

```NaN                687
G6                   4
B96 B98              4
C23 C25 C27          4```  
Interesting that there are a number of observations with multiple rooms recorded. Possible reasons - families, group bookings and so on. Isolate and observe the SibSp & Parch values for that group.


In [ ]:
# insert a cabin string length column
titanic_train.insert(
    # insert column after the Cabin column
    (titanic_train.columns.get_loc("Cabin") + 1), "Cabin_str_len", titanic_train.Cabin.str.len(), True
) 
# filter on that column string lengths greater than 5
multi_cabins = titanic_train.query("Cabin_str_len > 5")
# observe some summary stats
multi_cabins.describe()

All of the **multi_cabin** passengers are first class. SibSp and Parch both have minimum of 0.


In [ ]:
multi_cabins.SibSp.value_counts(dropna=False)


There are 6 rows with 0 siblings / spouses on board


In [ ]:
multi_cabins.Parch.value_counts(dropna=False)


There are 3 rows with 0 parents / children on board. Are there multi_cabin passengers with 0 for both Parch & SibSp?


In [ ]:
multi_cabins.query("SibSp == 0 and Parch == 0")

"Guggenheim boarded the Titanic at Cherbourg with his valet Victor Giglio and his "mistress" Mrs 
Aubart. Guggenheim and Giglio's ticket was 17593 and cost £79 4s1. Mr Guggenheim's chauffeur René 
Pernot travelled in second class.
Despite Etches best efforts Guggenheim soon returned to his room (B-82) and changed into his finest 
evening wear, his valet, Mr Giglio did likewise. He was later heard to remark 'We've dressed up in 
our best and are prepared to go down like gentlemen."  
https://www.encyclopedia-titanica.org/titanic-victim/benjamin-guggenheim.html  

There doesn't appear to be any supporting info as to why Frans Carlsson would have had multiple cabins associated, unlike Guggenheim who had an entourage. 
10% of passengers with multiple rooms are 0 for both family indicators.
I wonder how widespread the 'down with the ship' ethos was among the male passengers. Could look at
survival rates among males by age categories or classes?

In [ ]:
#tidy up
del multi_cabins

***

### Embarked

In [ ]:
# distribution?
titanic_train.Embarked.value_counts(dropna=False, normalize=True)

Interesting that the vast majority of passengers were boarded at Southampton. Let's look at the NaNs.

In [ ]:
titanic_train[titanic_train.Embarked.isna()]


Interesting, they both had no family on board but shared a ticket and cabin. Mrs Stone boarded the Titanic in Southampton on 10 April 1912 and was travelling in first class with her maid Amelie Icard. She occupied cabin B-28.
[Encyclopedia Titanica](https://www.encyclopedia-titanica.org/titanic-survivor/martha-evelyn-stone.html)

This could be evidence to amend the embarked column for Martha. Several sites online indicate this also. Safe to assume Amelie boarded at same port? The same website indicates Amelie boarded at S too.
[Encyclopedia Titanica](https://www.encyclopedia-titanica.org/titanic-survivor/amelia-icard.html)

In [ ]:
# replace the NaNs in this column with 'S'
titanic_train.Embarked = titanic_train.Embarked.fillna('S')

***

### Name

This column is used extensively with various simple pattern-matching techniques within the [Guide to feature engineering titanic data](https://triangleinequality.wordpress.com/2013/09/08/basic-feature-engineering-with-the-titanic-data/). 

By sorting on names it is possible to see patterns observed in the tickets that may indicate families. It could be possible to group family units together under group size. Insert a column for families with children.

In [ ]:
titanic_train.insert(
    # insert new column following duplicated_tickets
    (titanic_train.columns.get_loc('duplicated_tickets') + 1),
    # tickets are non-unique and have more than 0 Parent / children on board.
    # note the flaw in duplicated tickets - the tickets may be duplicated, but within the test set.
    'families_w_children', (titanic_train.duplicated_tickets > 1) & (titanic_train.Parch > 0)
)


In [ ]:

# insert a column for families with no children on board, as above with SibSp greater than 0 but 0 for Parch
titanic_train.insert(
     (titanic_train.columns.get_loc('families_w_children') + 1),
     'families_no_children',
     (titanic_train.duplicated_tickets > 1) & (titanic_train.SibSp > 0) & (titanic_train.Parch == 0)
)
# observe pairwise trends 
titanic_train.corr(method='pearson')



The engineered columns show trends with family indicators as expected, no sig trend with survival.

In [ ]:
# cache the engineered features to binary file for future use
pd.to_pickle(titanic_train, '../../cache/titanic_engineered.pkl')

***

## Training Dataset Distributions

In [ ]:
# What proportion of the total available data does the Kaggle test dataframe represent?

len(titanic_test) / len(titanic_train)

Nearly 50% training data. Higher than expected. 

***

### Pclass

In [ ]:
# compare Pclass
print(titanic_train.Pclass.value_counts(normalize=True))
titanic_test.Pclass.value_counts(normalize=True)

Slightly lower proportion 3rd class in test df.

***

### Sex

In [ ]:
# compare Sex ratios
print(titanic_train.Sex.value_counts(normalize=True))
titanic_test.Sex.value_counts(normalize=True)

1.1 % towards female in test df.

***

### Age


In [ ]:
# reset the plot canvas
plt.clf()
# Compare distributions of Age. Overlay histograms and set the opacity to 50%
plt.hist(titanic_train.Age, alpha=0.5, label='train')
plt.hist(titanic_test.Age, alpha=0.5, label='test')
# Place the legend in the best position
plt.legend(loc='upper right')


Distribution appears to be similar. Perhaps reverse proportions in ~20yrs vs ~30yrs. But overall likely to be of same distribution.


In [ ]:
print(titanic_train.Age.describe())
titanic_test.Age.describe()


Central tendency and distribution summary stats all very similar.

***

### SibSp

In [ ]:
# compare sibsp dists
# reset the plot canvas
plt.clf()
# Compare distributions of Sibsp
plt.hist(titanic_train.SibSp, alpha=0.5, label='train')
plt.hist(titanic_test.SibSp, alpha=0.5, label='test')
plt.legend(loc='upper right')

Visually analagous

In [ ]:
print(titanic_train.SibSp.describe())
titanic_test.SibSp.describe()

Quartiles are identical, similar means but std smaller in test dataset.

***

### Parch

In [ ]:
# reset the plot canvas
plt.clf()
# Compare distributions of Parch
plt.hist(titanic_train.Parch, alpha=0.5, label='train')
plt.hist(titanic_test.Parch, alpha=0.5, label='test')
plt.legend(loc='upper right')

Perhaps the test set here shows consistent skew to the right of training set, though analagous in proportion.

In [ ]:
print(titanic_train.Parch.describe())
titanic_test.Parch.describe()

Quartiles not influenced by the larger range observed within the test set

***

### Fare

In [ ]:
# Compare the distributions of the Fare feature in the training and test data. Use a visual method.
# reset the plot canvas
plt.clf()
# Compare distributions of Fare
plt.hist(titanic_train.Fare, alpha=0.5, label='train')
plt.hist(titanic_test.Fare, alpha=0.5, label='test')
plt.legend(loc='upper right')

Looks to be a few outlier high values in the test set, have they affected the central tendency?

In [ ]:
print(titanic_train.Fare.describe())
titanic_test.Fare.describe()

The quartiles are almost identical but the mean appears to have been influenced upwards by the high outliers within the test set.

***

### Embarked


In [ ]:
print(titanic_train.Embarked.value_counts(normalize=True))
titanic_test.Embarked.value_counts(normalize=True)

Rel. proportions of Southampton are 8% lower in test set.
